In [21]:
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import numpy
# randoml
from random import shuffle
# classifierfrom deepdist import DeepDist
from sklearn.linear_model import LogisticRegression

In [22]:
from deepdist import DeepDist

In [23]:
from pyspark import SparkContext
sc = SparkContext()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=yarn) created by __init__ at <ipython-input-3-33ce3f59c0b1>:2 

In [24]:
corpus =sc.textFile('hdfs:///user/hadoop/data/tagged_docs_shuffled.txt').map(
    lambda s: TaggedDocument(s[19:].split(), [s[:18]]))

In [25]:
def gradient(model, taggedData):  # executes on workers
#     print type(taggedData)
#     sentence = []
#     for i in taggedData:
#         sentence.append(i)
    syn0, syn1, doctag = model.wv.syn0.copy(), model.syn1neg.copy(), model.docvecs.vectors_docs.copy()
    model.train(taggedData,total_examples=model.corpus_count,epochs=3)
    return {'syn0': model.wv.syn0 - syn0,'syn1':model.syn1neg-syn1,'doctag':model.docvecs.vectors_docs-doctag}

def descent(model, update): # executes on master
    #print model.docvecs.doctag_syn0.shape
    #print update['doctag'].shape
    model.wv.syn0 += update['syn0']
    model.syn1neg += update['syn1']
    model.docvecs.vectors_docs += update['doctag']
    #print update['taggedDoc'].shape,

In [27]:
%%time
collectedCorpus = corpus.collect()

CPU times: user 9.23 s, sys: 1.15 s, total: 10.4 s
Wall time: 25.5 s


In [28]:
%%time
doc2VecModel = Doc2Vec(collectedCorpus)

CPU times: user 8min 17s, sys: 16.7 s, total: 8min 34s
Wall time: 4min 49s


In [40]:
%%time
with DeepDist(doc2VecModel,master='192.168.91.1:5000') as dd:
    for i in range(3):
        dd.train(corpus,gradient,descent)

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)


Listening to 0.0.0.0:5000...

*** Master: 192.168.91.1:5000



192.168.91.3 - - [24/Apr/2018 11:50:52] "GET /model HTTP/1.1" 200 -
192.168.91.2 - - [24/Apr/2018 11:50:52] "GET /model HTTP/1.1" 200 -
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  del sys.path[0]
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `syn1neg` (Attribute will be removed in 4.0.0, use self.trainables.syn1neg instead).
  
192.168.91.2 - - [24/Apr/2018 11:51:32] "POST /update HTTP/1.1" 200 -
192.168.91.3 - - [24/Apr/2018 11:51:33] "POST /update HTTP/1.1" 200 -



*** Master: 192.168.91.1:5000



192.168.91.3 - - [24/Apr/2018 11:51:36] "GET /model HTTP/1.1" 200 -
192.168.91.2 - - [24/Apr/2018 11:51:36] "GET /model HTTP/1.1" 200 -
192.168.91.3 - - [24/Apr/2018 11:52:16] "POST /update HTTP/1.1" 200 -
192.168.91.2 - - [24/Apr/2018 11:52:16] "POST /update HTTP/1.1" 200 -



*** Master: 192.168.91.1:5000



192.168.91.3 - - [24/Apr/2018 11:52:19] "GET /model HTTP/1.1" 200 -
192.168.91.2 - - [24/Apr/2018 11:52:20] "GET /model HTTP/1.1" 200 -
192.168.91.3 - - [24/Apr/2018 11:52:59] "POST /update HTTP/1.1" 200 -
192.168.91.2 - - [24/Apr/2018 11:53:00] "POST /update HTTP/1.1" 200 -
192.168.91.1 - - [24/Apr/2018 11:53:00] "POST /shutdown HTTP/1.1" 200 -


Exit requested...
CPU times: user 9.72 s, sys: 3.01 s, total: 12.7 s
Wall time: 2min 12s


In [42]:
train_arrays = numpy.zeros((25000, 100))
train_labels = numpy.zeros(25000)
for i in range(12500):
    prefix_train_pos = 'TRAIN_POS' + "%09d"%(i+1,)
    prefix_train_neg = 'TRAIN_NEG' + "%09d"%(i+1,)
    train_arrays[i] = dd.model[prefix_train_pos]
    train_arrays[12500 + i] =dd.model[prefix_train_neg]
    train_labels[i] = 1
    train_labels[12500 + i] = 0
test_arrays = numpy.zeros((25000, 100))
test_labels = numpy.zeros(25000)
for i in range(12500):
    prefix_test_pos = 'TEST__POS' + "%09d"%(i+1,)
    prefix_test_neg = 'TEST__NEG' + "%09d"%(i+1,)
    test_arrays[i] = dd.model[prefix_test_pos]
    test_arrays[12500 + i] = dd.model[prefix_test_neg]
    test_labels[i] = 1
    test_labels[12500 + i] = 0
classifier = LogisticRegression()
classifier.fit(train_arrays, train_labels)

print classifier.score(test_arrays, test_labels)

0.843


In [41]:
a = dd.model.most_similar(positive=['king','woman'],negative=['man'])
map(lambda x: x[0],a)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[u'queen',
 u'prince',
 u'empress',
 u'publisher',
 u'donnelly',
 u'misa',
 u'maid',
 u'diddy',
 u'gellar',
 u'westley']